In [1]:
import os

imdb_dir = r'C:/Users/clare\Documents/aclImdb'
train_dir = os.path.join(imdb_dir, 'train')

labels = []
texts = []

for label_type in ['neg', 'pos']:
    dir_name = os.path.join(train_dir, label_type)
    for fname in os.listdir(dir_name):
        if fname[-4:] == ".txt":
            f = open(os.path.join(dir_name, fname), encoding='utf8')
            texts.append(f.read())
            f.close()
            if label_type == 'neg':
                labels.append(0)
            else:
                labels.append(1)

print(len(texts))

25000


In [2]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

maxlen = 100
training_samples = 200
validation_samples = 10000
max_words = 10000

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index

print('共用了 %s 個 token 字詞.' % len(word_index))

data = pad_sequences(sequences, maxlen=maxlen)
labels = np.asarray(labels)

print("資料張量：", data.shape)
print("標籤張量：", labels.shape)

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

x_train = data[:training_samples]
y_train = labels[:training_samples]
x_val = data[training_samples: training_samples + validation_samples]
y_val = labels[training_samples: training_samples + validation_samples]

共用了 88582 個 token 字詞.
資料張量： (25000, 100)
標籤張量： (25000,)


In [8]:
glove_dir = r'C:/Users/clare/Documents'

embeddings_index = {}
f = open(os.path.join(glove_dir, r'glove.6B.100d.txt'), encoding='UTF-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print("共有 %s 個嵌入向量." % len(embeddings_index))

共有 400000 個嵌入向量.


In [9]:
embedding_dim = 100
embedding_matrix = np.zeros((max_words, embedding_dim))
print(embedding_matrix.shape)
for word, i in word_index.items():
    if i < max_words:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

(10000, 100)


In [10]:
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense

model = Sequential()

model.add(Embedding(max_words, embedding_dim, input_length=maxlen))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 100)          1000000   
_________________________________________________________________
flatten (Flatten)            (None, 10000)             0         
_________________________________________________________________
dense (Dense)                (None, 32)                320032    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 1,320,065
Trainable params: 1,320,065
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False

In [12]:
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])
history = model.fit(x_train, y_train, epochs=10,
                    batch_size=32, validation_data=(x_val, y_val))
model.save_weights('pre_trained_glove_model.h5')

Epoch 1/10
7/7 [==============================] - 1s 104ms/step - loss: 1.8264 - acc: 0.4600 - val_loss: 0.7892 - val_acc: 0.5008
Epoch 2/10
7/7 [==============================] - 1s 80ms/step - loss: 0.6672 - acc: 0.6400 - val_loss: 0.6880 - val_acc: 0.5551
Epoch 3/10
7/7 [==============================] - 1s 75ms/step - loss: 0.3675 - acc: 0.8750 - val_loss: 1.2133 - val_acc: 0.5006
Epoch 4/10
7/7 [==============================] - 1s 74ms/step - loss: 0.3246 - acc: 0.9150 - val_loss: 0.6841 - val_acc: 0.5739
Epoch 5/10
7/7 [==============================] - 1s 76ms/step - loss: 0.2762 - acc: 0.9300 - val_loss: 0.9045 - val_acc: 0.5151
Epoch 6/10
7/7 [==============================] - 0s 70ms/step - loss: 0.1502 - acc: 0.9900 - val_loss: 0.7363 - val_acc: 0.5585
Epoch 7/10
7/7 [==============================] - 1s 71ms/step - loss: 0.1044 - acc: 0.9950 - val_loss: 1.9804 - val_acc: 0.5004
Epoch 8/10
7/7 [==============================] - 1s 73ms/step - loss: 0.2018 - acc: 0.9200 - va

dict_keys(['loss', 'acc', 'val_loss', 'val_acc'])

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validaion acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.figure()